In [27]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from package.logger import Timed, rlog, setup
setup("INFO")


from package import storage
import pandas as pd
from package import strtime
import folium
import os
from package.osm import osm
from package.geometa import GeoMeta

In [29]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)


geo_meta = GeoMeta.load("../data/geometa.pkl")

with Timed.info("Reading stops"):
	other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_boundary(osm_reader, geo_meta)

[13:49:20] INFO     Reading stops                                                                   ]8;id=127574;file:///tmp/ipykernel_45243/2252360690.py\2252360690.py]8;;\:]8;id=941399;file:///tmp/ipykernel_45243/2252360690.py#8\8]8;;\

           INFO     Reading stops done (0.07 seconds)                                               ]8;id=720035;file:///tmp/ipykernel_45243/2252360690.py\2252360690.py]8;;\:]8;id=334110;file:///tmp/ipykernel_45243/2252360690.py#8\8]8;;\

           INFO     Using existing OSM data                                                        ]8;id=640759;file:///tmp/ipykernel_45243/2252360690.py\2252360690.py]8;;\:]8;id=308329;file:///tmp/ipykernel_45243/2252360690.py#15\15]8;;\

           INFO     Getting OSM graph                                                              ]8;id=714968;file:///tmp/ipykernel_45243/2252360690.py\2252360690.py]8;;\:]8;id=568692;file:///tmp/ipykernel_45243/2252360690.py#19\19]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=994880;file:///tmp/ipykernel_45243/2252360690.py\2252360690.py]8;;\:]8;id=290111;file:///tmp/ipykernel_45243/2252360690.py#19\19]8;;\

[13:49:21] INFO     Getting OSM graph done (0.59 seconds)                                          ]8;id=998022;file:///tmp/ipykernel_45243/2252360690.py\2252360690.py]8;;\:]8;id=633830;file:///tmp/ipykernel_45243/2252360690.py#19\19]8;;\

In [30]:
path_source = "/home/moritz/dev/uni/mcr-py/data/bags.pkl"

data = storage.read_any_dict(path_source)
path_manager = data["path_manager"]
node_map = data["multi_modal_node_to_resetted_map"]
walking_node_map = data["walking_node_to_resetted_map"]
reverse_node_map = {v: k for k, v in node_map.items()}
reverse_walking_node_map = {v: k for k, v in walking_node_map.items()}
stops_df = data["stops_df"]
bags_i = data["bags_i"]

In [31]:
labels = pd.DataFrame(
    [
        (label.node_id, label.values[0], label.values[1], n_transfers, label)
        for n_transfers, bags in bags_i.items()
        for bag in bags.values()
        for label in bag
    ],
    columns=["osm_node_id", "time", "cost", "n_transfers", "label"],
)

labels["human_readable_time"] = labels["time"].apply(strtime.seconds_to_str_time)
labels = labels.sort_values("n_transfers")
labels = labels.drop_duplicates(subset=["osm_node_id", "time", "cost"], keep="first")
labels.drop(columns=["label"])

,osm_node_id,time,cost,n_transfers,human_readable_time
0,6684028881,28933,0,0,08:02:13
102,8793169852,28841,0,0,08:00:41
101,10098069490,29008,0,0,08:03:28
100,9049263570,28991,0,0,08:03:11
99,4778885168,28999,0,0,08:03:19
...,...,...,...,...,...
47,8908240206,28882,0,0,08:01:22
48,2547316860,28877,0,0,08:01:17
49,9049263562,28983,0,0,08:03:03
43,4778885551,28995,0,0,08:03:15


In [32]:
from package.minute_city import minute_city

In [33]:
pois = minute_city.fetch_pois_for_area(geo_meta.boundary, nodes)

In [34]:
labels["target_id_osm"] = labels["osm_node_id"]
labels["start_id_hex"] = "aaa"

In [35]:
poi_labels = minute_city.add_pois_to_labels(labels, pois)

In [36]:
# types = ["grocery", "education", "health", "banks", "parks", "sustenance", "shops"]
types = list(pois["type"].unique())

In [37]:
profiles_df = minute_city.get_profiles_df(poi_labels, types)

[13:49:28] INFO     Grouping labels                                                               ]8;id=30027;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=75212;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#58\58]8;;\

           INFO     Grouping labels done (0.01 seconds)                                           ]8;id=381712;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=418548;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#58\58]8;;\

           INFO     Calculating profiles                                                          ]8;id=533931;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=515972;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#66\66]8;;\

  0%|          | 0/1 [00:00<?, ?it/s]

[13:49:29] INFO     Calculating profiles done (0.30 seconds)                                      ]8;id=76976;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=121095;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#66\66]8;;\

           INFO     Creating profiles dataframe                                                   ]8;id=127281;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=492463;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#77\77]8;;\

           INFO     Creating profiles dataframe done (0.01 seconds)                               ]8;id=39864;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py\minute_city.py]8;;\:]8;id=913212;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/minute_city/minute_city.py#77\77]8;;\

In [38]:
profiles_df

,cost_0,any_column_different,required_cost_for_optimal
hex_id,,,
aaa,292,False,0


In [39]:
max_time = strtime.str_time_to_seconds("09:00:00")
max_cost = 1000
max_transfers = 10

valid_labels = labels[
	(labels["time"] <= max_time)
	& (labels["cost"] <= max_cost)
	& (labels["n_transfers"] <= max_transfers)
]

In [40]:
selection = valid_labels.sample(min(1000, len(valid_labels)))

max_cost = selection.cost.max()

from typing import Dict
from colorsys import hls_to_rgb

def hsl_to_hex(h: float, l: float, s: float) -> str:
    r, g, b = [int(x * 255.0) for x in hls_to_rgb(h, l, s)]
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

def generate_colors_hsl(max_cost: float) -> Dict[int, str]:
    if max_cost == 0:
        return {0: "#ffcccc"}
    return {
        cost: hsl_to_hex(h=0, l=0.5 + 0.4 * (max_cost - cost) / max_cost, s=0.8)
        for cost in range(int(max_cost) + 1)
    }

colors = generate_colors_hsl(max_cost)

In [41]:
start_node_id = labels[labels.time == labels.time.min()].iloc[0].osm_node_id

In [46]:
bicycles = storage.read_df("../data/bicycle_locations/2022-11-01_09_01_00.csv")
bicycles

,lat,lon
0,50.985966,7.031896
1,50.985966,7.031896
2,50.860863,7.037264
3,51.036499,6.882667
4,50.899956,7.018377
...,...,...
1493,50.975285,7.007419
1494,50.959141,7.008940
1495,50.944199,7.001608
1496,50.964500,6.953253


In [49]:
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index

start_node = nodes_by_id.loc[start_node_id]

m = folium.Map(location=[start_node.lat, start_node.lon], zoom_start=13)

# for label in selection.itertuples():
for label in labels.itertuples():
	node = nodes_by_id.loc[label.osm_node_id]
	folium.CircleMarker(
		location=[node.lat, node.lon],
		radius=3,
		weight=1,
		fill=True,
		fill_color=colors[label.cost],
		fill_opacity=1,
		color="black",
		popup=f"{label.human_readable_time} ({label.cost})",
	).add_to(m)

folium.CircleMarker(
	location=[start_node.lat, start_node.lon],
	radius=5,
	weight=1,
	fill=True,
	fill_color="green",
	fill_opacity=1,
	color="black",
	popup=f"Start",
).add_to(m)

for bicycle in bicycles.itertuples():
	folium.CircleMarker(
		location=[bicycle.lat, bicycle.lon],
		radius=3,
		weight=1,
		fill=True,
		fill_color="blue",
		fill_opacity=1,
		color="black",
		popup=f"Bicycle",
	).add_to(m)

m

In [43]:
from package.mcr.path import Path, GTFSPath, PathType

translator_map = {
    PathType.WALKING: reverse_walking_node_map,
    PathType.CYCLING_WALKING: reverse_node_map,
    PathType.PUBLIC_TRANSPORT: None,
}
no_prefix_reverse_walking_node_map = {
    k: int(v[1:]) for k, v in reverse_walking_node_map.items()
}
no_prefix_reverse_node_map = {k: int(v[1:]) for k, v in reverse_node_map.items()}
no_prefix_translator_map = {
    PathType.WALKING: no_prefix_reverse_walking_node_map,
    PathType.CYCLING_WALKING: no_prefix_reverse_node_map,
    PathType.PUBLIC_TRANSPORT: None,
}

In [44]:
walking_result_bags = bags_i[1]

KeyError: 1

In [ ]:
walking_result_bags_flat = []
for node_id, bag in walking_result_bags.items():
	for label in bag:
		walking_result_bags_flat.append((node_id, label))

In [ ]:
path_objs_with_ids = pd.Series(
    list(
        map(
            lambda x: (
                x[0],
                path_manager.reconstruct_and_translate_path_for_label(
                    x[1], no_prefix_translator_map
                ),
            ),
            walking_result_bags_flat[:10000],
        )
    )
)
node_ids, path_objs = list(zip(*path_objs_with_ids))
path_objs[:3]

([Path(path_type=PathType.WALKING, path=[394001227, 394001227, 2427289539, 3141997559, 394001505, 279032270, 8805331132, 4394977416, 2139170766, 649698710, 7018921111, 7018921108, 256201390], meta={'values': [28974, 0], 'hidden_values': [0, 0]}),
  Path(path_type=PathType.CYCLING_WALKING, path=[2340562483, 2340562483, 10712755234, 7803048298, 1518107637, 652791048, 8634843143, 8634843144], meta={'values': [29020, 0], 'hidden_values': [0, 0]}),
  Path(path_type=PathType.WALKING, path=[], meta={'values': [29020, 0], 'hidden_values': [0, 0]})],
 [Path(path_type=PathType.WALKING, path=[394001227, 394001227, 2427289539, 3141997559, 394001505, 279032270, 8805331132, 4394977416, 2139170766, 649698710, 7018921111, 7018921108, 256201390], meta={'values': [28974, 0], 'hidden_values': [0, 0]}),
  Path(path_type=PathType.CYCLING_WALKING, path=[2340562483, 10712755234, 7803048298, 1518107637, 652791048, 8634843143, 8634843144, 8634843144], meta={'values': [28989, 100], 'hidden_values': [15, 0]}),
 

In [ ]:
labels.drop(columns=["label"])

,osm_node_id,time,cost,n_transfers,human_readable_time,target_id_osm,start_id_hex
0,943957934,29163,0,0,08:06:03,943957934,aaa
715,2427289540,28975,0,0,08:02:55,2427289540,aaa
716,394001505,28813,0,0,08:00:13,394001505,aaa
717,9975803558,29176,0,0,08:06:16,9975803558,aaa
718,664174298,29184,0,0,08:06:24,664174298,aaa
...,...,...,...,...,...,...,...
18178,7022495313,31200,320,5,08:40:00,7022495313,aaa
18635,465623830,33060,320,5,09:11:00,465623830,aaa
18417,7431312249,31800,320,5,08:50:00,7431312249,aaa
18418,7431312249,31020,420,5,08:37:00,7431312249,aaa


In [ ]:
import random
i = random.sample(list(labels.osm_node_id.unique()), 1)[0]
# i = 3922625314
print(i)
selection = labels[labels.osm_node_id == i]
selection.drop(columns=["label"])

6180436678


,osm_node_id,time,cost,n_transfers,human_readable_time,target_id_osm,start_id_hex
4059,6180436678,29386,0,1,08:09:46,6180436678,aaa
4060,6180436678,29106,100,1,08:05:06,6180436678,aaa
7299,6180436678,29106,200,2,08:05:06,6180436678,aaa


In [ ]:
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
start_time = strtime.str_time_to_seconds("08:00:00")

In [ ]:
def format_meta(meta, previous_meta):
	values = meta["values"]
	arrival_time = values[0]
	cost = values[1]

	if previous_meta:
		previous_values = previous_meta["values"]
		previous_arrival_time = previous_values[0]
		previous_cost = previous_values[1]
		
		arrival_time -= previous_arrival_time
		cost -= previous_cost
	else:
		arrival_time -= start_time

	return f"{strtime.seconds_to_str_time(arrival_time)} ({cost})"

In [ ]:
toloop = selection

stops_by_id = stops_df.set_index("stop_id")
sample_label = selection.iloc[0]
sample_node_id = sample_label.osm_node_id
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index
sample_node = nodes_by_id.loc[sample_node_id]

m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=13)

for row in toloop.itertuples():
    label = row.label
    end_node_id = row.osm_node_id
    end_node = nodes_by_id.loc[end_node_id]

    folium.CircleMarker(
        location=[end_node.lat, end_node.lon],
        popup=f"End: {end_node_id}",
        color="red",
        radius=3,
    ).add_to(m)

    paths = path_manager.reconstruct_and_translate_path_for_label(label, translator_map)
    for i, path in enumerate(paths):
        if isinstance(path, Path):
            if path.path == []:
                continue
            cycling_path_nodes = [
                nodes_by_id.loc[int(node_id[1:])]
                for node_id in path.path
                if node_id[0] == "B"
            ]
            walking_path_nodes = [
                nodes_by_id.loc[int(node_id[1:])]
                for node_id in path.path
                if node_id[0] == "W"
            ]
            path_lat_lon = [(node.lat, node.lon) for node in cycling_path_nodes]
            previous_meta = paths[i - 1].meta if i > 0 else None
            meta = format_meta(path.meta, previous_meta)
            if path_lat_lon != []:
                folium.PolyLine(
                    path_lat_lon, color="blue", weight=2, popup=str(meta)
                ).add_to(m)
            path_lat_lon = [(node.lat, node.lon) for node in walking_path_nodes]
            if path_lat_lon != []:
                folium.PolyLine(
                    path_lat_lon, color="red", weight=2, popup=str(meta)
                ).add_to(m)
        elif isinstance(path, GTFSPath):
            start_stop_id = path.start_stop_id
            end_stop_id = path.end_stop_id
            start_stop = stops_by_id.loc[start_stop_id]
            end_stop = stops_by_id.loc[end_stop_id]
            trip = path.trip_id
            if len(trip) >= 10:
                trip = trip[:10] + "..."

            previous_meta = paths[i - 1].meta if i > 0 else None
            line_msg = f"Trip: {trip}\n---\n {format_meta(path.meta, previous_meta)}"

            path_lat_lon = [
                (float(start_stop.stop_lat), float(start_stop.stop_lon)),
                (float(end_stop.stop_lat), float(end_stop.stop_lon)),
            ]
            folium.PolyLine(
                path_lat_lon,
                color="green",
                weight=2,
                popup=line_msg,
            ).add_to(m)

            folium.CircleMarker(
                location=[float(start_stop.stop_lat), float(start_stop.stop_lon)],
                popup=f"Start: {start_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
            folium.CircleMarker(
                location=[float(end_stop.stop_lat), float(end_stop.stop_lon)],
                popup=f"End: {end_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
        else:
            raise Exception("Unknown path type")

m
